In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import time
import pickle
from tqdm import tqdm
# for some reason, need to go to the sheaf_kg directory in order for torch.load to work
os.chdir('/home/gebhart/projects/sheaf_kg/sheaf_kg')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pykeen
import torch
from pykeen.pipeline import pipeline
from train_sheafE_betae import read_dataset, dataset_to_device
import torch
from torch_geometric.data import NeighborSampler, Data, ShaDowKHopSampler, \
                                GraphSAINTRandomWalkSampler, RandomNodeSampler, \
                                ClusterData, ClusterLoader

In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
dataset = 'FB15k-237'
use_section = 0
device = 'cpu'
train_test_queries = 'test'
model_name = 'SheafE_Complex_Queries_16embdim_16esdim_1sec_2norm_1epochs_SoftplusLoss_20210424-1401'
save_loc = '/home/gebhart/projects/sheaf_kg/data/{}/{}/trained_model.pkl'.format(dataset,model_name)
dataset_loc = '/home/gebhart/projects/sheaf_kg/data/{}-betae'.format(dataset)

In [4]:
datasets = read_dataset(dataset_loc)
datasets = dataset_to_device(datasets, device)

In [5]:
ds = pykeen.datasets.get_dataset(dataset=dataset, dataset_kwargs=dict(create_inverse_triples=False))
training = ds.training.mapped_triples
# 237*8*16*8*16/training.shape[0]

You're trying to map triples with 30 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 28 from 20466 triples were filtered out


In [6]:
training

tensor([[    0,   120, 13647],
        [    0,   125, 13647],
        [    0,   126,     0],
        ...,
        [14503,   126, 14503],
        [14503,   127, 12940],
        [14504,   127, 12940]])

In [7]:
from ogb.nodeproppred import PygNodePropPredDataset
d = PygNodePropPredDataset('ogbn-products')

In [8]:
data = d[0]
split_idx = d.get_idx_split()
train_idx = split_idx['train']

In [9]:
training.shape

torch.Size([272115, 3])

In [10]:
data.edge_index[0].max()

tensor(2449028)

In [11]:
data

Data(edge_index=[2, 123718280], x=[2449029, 100], y=[2449029, 1])

In [63]:
edge_index = torch.empty((2,training.shape[0]), dtype=torch.int64)
edge_index[0,:] = training[:,0]
edge_index[1,:] = training[:,2]
train_idx = torch.unique(edge_index.flatten())
train_loader = NeighborSampler(edge_index, node_idx=train_idx,
                               sizes=[3,2,1], batch_size=1,
                               shuffle=True, num_workers=2)
i = 0
for batch_size, n_id, adjs in train_loader:
    e_ids = []
    print(adjs)
    for j, (eix, e_id, size) in enumerate(adjs):
        e_ids.append(e_id)
    e_ids = torch.cat(e_ids, dim=0)
    print(training[e_ids])
    i += 1
    if i > 10:
        break

[EdgeIndex(edge_index=tensor([[ 1,  7, 12, 11, 13, 14, 15, 16, 17, 18, 19, 20],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]]), e_id=tensor([229683, 215493,  87789, 175510, 205908, 219409,  25816, 146805,  62459,
         13084, 266625, 124963]), size=(21, 12)), EdgeIndex(edge_index=tensor([[ 4,  5,  6,  7,  8,  9, 10, 11],
        [ 0,  0,  1,  1,  2,  2,  3,  3]]), e_id=tensor([191549,  68076, 220040, 215486,  42783, 193566,  59950, 175510]), size=(12, 4)), EdgeIndex(edge_index=tensor([[1, 2, 3],
        [0, 0, 0]]), e_id=tensor([229723, 152004,  53260]), size=(4, 1))]
tensor([[11943,    11,  7198],
        [11198,    49, 11943],
        [ 4773,     9,  8109],
        [ 9254,    32,  2921],
        [10761,    93, 10033],
        [11417,    93,  3776],
        [ 1331,   190, 11446],
        [ 7845,    32, 11198],
        [ 3471,    11,  2371],
        [  701,    10, 10141],
        [14007,    12,  3338],
        [ 6642,    10,  9254],
        [10033,   100,  7198],
       

In [13]:
edge_index.shape

torch.Size([2, 272115])

In [14]:
object_data = Data(edge_index=edge_index)
object_data.num_nodes = train_idx.shape[0]

In [34]:
saint_loader = GraphSAINTRandomWalkSampler(object_data, batch_size=1, walk_length=3,
                                     num_steps=100, num_workers=3, shuffle=True)
i = 0
for data in saint_loader:
    print(data.edge_index, data)
    i += 1
    if i > 4:
        break

tensor([[0, 2, 3],
        [1, 0, 2]]) Data(edge_index=[2, 3])
tensor([[0, 1, 1, 2, 2, 2, 2],
        [2, 2, 2, 0, 0, 1, 1]]) Data(edge_index=[2, 7])
tensor([[0, 0, 0, 1, 1, 2, 2, 2, 2, 3, 3],
        [1, 2, 3, 0, 2, 0, 0, 1, 1, 0, 0]]) Data(edge_index=[2, 11])
tensor([[0],
        [2]]) Data(edge_index=[2, 1])
tensor([[0, 1, 3],
        [2, 2, 1]]) Data(edge_index=[2, 3])


In [33]:
object_data.edge_index

tensor([[    0,     0,     0,  ..., 14503, 14503, 14504],
        [13647, 13647,     0,  ..., 14503, 12940, 12940]])

In [62]:
node_loader = RandomNodeSampler(object_data, num_parts=200, num_workers=3, shuffle=True)
i = 0
for data in node_loader:
    print(data.edge_index, data)
    i += 1
    if i > 10:
        break

tensor([[ 6, 37, 75, 78, 79],
        [ 6, 67, 75, 78, 79]]) Data(edge_index=[2, 5])
tensor([[ 0,  4,  8,  8,  9, 10, 29, 36, 36, 41, 54, 61, 77, 78, 79],
        [ 0, 70,  8,  8,  9, 10, 29, 36, 36, 26, 54,  4, 77, 78, 79]]) Data(edge_index=[2, 15])
tensor([[ 1, 14, 14, 16, 16, 31, 44, 47, 50, 53],
        [47, 14, 14, 26, 31, 46, 57,  1, 50, 53]]) Data(edge_index=[2, 10])
tensor([[ 5,  5, 15, 21, 21, 23, 38, 44, 44, 46, 52, 53],
        [ 5,  5, 53, 21, 21, 50,  7, 44, 44, 46, 47, 15]]) Data(edge_index=[2, 12])
tensor([[ 5,  5,  7,  7,  7,  7, 11, 15, 15, 17, 17, 21, 29, 32, 34, 34, 34, 34,
         38, 44, 48, 51, 51, 54, 58, 58, 59, 59, 66, 68],
        [34, 58, 34, 34, 38, 45, 51, 15, 15, 17, 17, 51, 51, 23,  7, 58, 58, 58,
         64, 57, 48, 45, 59, 54, 34, 34, 51, 51, 66, 68]]) Data(edge_index=[2, 30])
tensor([[10, 10, 16, 16, 17, 27, 40, 44, 46, 60, 68, 68, 71, 71, 82],
        [10, 10, 16, 16, 60, 37,  6, 44, 68, 17, 63, 63, 82, 82, 71]]) Data(edge_index=[2, 15])
tensor([[11

In [26]:
training[71]

tensor([   13,   191, 13720])

In [17]:
cluster_data = ClusterData(object_data, num_parts=50, recursive=False)
cluster_loader = ClusterLoader(cluster_data, batch_size=1, shuffle=True, num_workers=3)
for data in cluster_loader:
    print(data.edge_index, data)
    i += 1
    if i > 4:
        break

Computing METIS partitioning...
Done!
tensor([[  0,   0,   0,  ..., 280, 281, 281],
        [ 18,  20,  23,  ..., 231, 218, 281]]) Data(edge_index=[2, 994])
